Cryptography

### Team
<ol>
    <li> Mark Pearson </li>
    <li> Ronan Geraghty</li>
    <li> Milan Patel </li>
    <li> Will Steynor </li>
</ol>



In [3]:
from NumberFunctions import *
import numpy as np
import random as ran
import matplotlib.pyplot as plt
from time import process_time
import pandas as pd
import numba
import time

Stuff extra to include

+ difference between asymmetric and symmetric cryptography

+ Steganography

# Core 2: RSA Signature Message

## Modular Inverse

Solve, 
> $ax = 1 \pmod{m} $ for  $x \in \{1,2,3,...,m-1\}$


Proof

We have, where $x,y \in \mathbb{Z}$ are calculated by the Euclidean algorithm.
> $ax+by = gcd(a,b)$

We only need to take into account when $gcd(a,b)=1$ as $e$ is chosen so $gcd(e,\phi{N})=1$

Let $m=b$, where $gcd(a,m) = 1$

So we get
> \begin{align*}
ax+my &= 1\\
ax + my &\equiv 1\pmod{m}\\
ax &\equiv 1\pmod{m}
\end{align*}

Therefore, $x$ is a modular inverse of $a$ for $m$

However, by using the Euclidean algorithm sometimes $x$ is not in the set $\{1,2,...,m-1\}$. This is a problem as if $x$ is negative when we use it later for the RSA signature we need to take the modulo of $int^{x}$, to do this we must have $int = int^{x}$

Let us consider the case where $x \notin \{1,2,...,m-1\}$, Let $d \in \mathbb{Z}$
> \begin{align*}
ax &\equiv 1\pmod{m}\\
ax + adm &\equiv 1\pmod{m}\\
a(x + dm) &\equiv 1\pmod{m}
\end{align*}

Set $x\prime = x + dm \iff x\prime = x \pmod{m}$

We know $x\prime \in \{1,2,...,m-1\}$


In [4]:
@numba.jit('int64(int64)', nopython=True)
def totient_fast(n):
    '''return the totient of n,
        number of postive integers < n which are coprime to n.'''
    num = 0
    for i in range(1, n):
        if gcd_fast(i, n) == 1:
            num += 1
    return num

@numba.vectorize('int64(int64)', nopython= True, target='parallel')
def totient_fast_ufunc_par(n):
    return totient_fast(n)

def totient(n):
    '''return the totient of n,
        number of postive integers < n which are coprime to n.'''
    num = 0
    for i in range(1, n):
        if gcd_fast(i, n) == 1:
            num += 1
    return num




def my_hash(message, N):
    '''Turn message to bits
    Then turn into int h'''
    return hash(message) % N


def modular_inverse(a, n):
    '''Solves ax = 1 mod(n) for x in {1,2,3,...,m-1}'''
    gcd, x, y = gcd_ext(a, n)
    if gcd == 1:
        return x % n
    else:
        return "gcd not = 1"

def get_primes_composites(n, is_prime_FUN=isprime_basic):
    '''returns a numpy array of primes and composites with largest number being n,
    is_prime_FUN: which function will be used to identify if a number is prime'''
    primes = []
    composites = []

    for num in range(1, n+1):
        if is_prime_FUN(num):
            primes.append(num)
        else:
            composites.append(num)

    return np.array(primes), np.array(composites)


def primality_test(N, t):
    for i in range(0, t):
        a = ran.randrange(0, N-1)
        if a**(N-1) % N != 1:
            return "composite"
    return "prime"

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'gcd_fast':[0m [1m[1mcannot determine Numba type of <class 'numba.ir.UndefinedType'>[0m
[1m
File "<ipython-input-4-bf6b043b3952>", line 7:[0m
[1mdef totient_fast(n):
    <source elided>
    for i in range(1, n):
[1m        if gcd_fast(i, n) == 1:
[0m        [1m^[0m[0m
[0m

In [ ]:
# Tests

assert totient(7) == 6
assert totient(35) == 24
assert totient(5321) == 4992

assert max(totient_fast_ufunc_par(np.array([7,35,5321])) - np.array([6,24,4992]))<1e-10

assert modular_inverse(15, 7) == 1

a = 3487
n = 9837
b = modular_inverse(a, n)
assert (a*b) % n == 1

# This test gets negative value originally, but inverse needs to be inbetween {1,2,..,m-1}
a = 3231
n = 12334
assert modular_inverse(a, n) == 11895


expected_primes = np.array([2, 3, 5, 7, 11, 13, 17, 19])
expected_composites = np.array([1, 4, 6, 8, 9, 10, 12, 14, 15, 16, 18, 20])

primes, composites = get_primes_composites(20)
assert max(primes - expected_primes) == 0
assert max(composites - expected_composites) == 0

## Generating primes using Miller Rabin

An n bit number is one which requires n bits to be stored, with the most significant bit (the left most bit) being 1. This means the n bit number is in the range $[2^{n-1}, 2^n)$.

we need to generate primes of n bit length, the idea is to pick a n bit number and check if it is prime. However, with large values of primes the traditional method of checking if numbers divide the prime doesn't work very fast, if n is prime it has at best a speed of $O(\sqrt{n})$. 

However, by using a probabilistic approach to find primes. Large primes can be found. The longer the algorithm is run for the higher the probability of getting a prime is. We will be using the probabilistic method called Miller Rabin.

### How Miller Rabin Works

Let $n$ be prime with $n>2$

So we have $n-1$ must be even, this means it can be written as $2^sd$ with $s,d \in \mathbb{N}_+$ and $d$ odd.

Therefore, we have one of these must be true for each $a\in \{2,3,...,n-1\}$

> $a^d \equiv 1 \mod{n}$

or

>$a^{2^rd} \equiv -1 \mod{n}$ for $r \in \{0,1,2,...,s-1\}$

To prove this we will use Fermat's Little Theorem which states for a prime number $n$ and $a\in \mathbb{Z}/n\mathbb{Z}$.
> $a^{n-1} \equiv 1\mod{n} \iff a^{n} \equiv a \mod{n}$

This can be written as
>$a^{n-1} = a^{2^sd}$

Then by taking square roots
>\begin{align*}
\sqrt{a^{2^sd}} &\equiv \sqrt{1} \mod{n}\\
&\text{ }\\
a^{2^{s-1}d} &\equiv +1 \mod{n} \\
\textrm{or}&\\
a^{2^{s-1}d} &\equiv -1 \mod{n}
\end{align*}

If it is $+1$ we square root again, when we get the modulo equal to $-1$ we can no longer take any more square roots so that equation is true. Therefore, we get one of the above equations must be true.

However, we will be using the contrapositive of the above claim,

The contrapositive states,
> $(P \rightarrow Q) \iff (\neg{Q} \rightarrow \neg{P})$

So we get the contrapositive of the above claim is 
> $a^d \not\equiv 1 \mod{n}$

and

>$a^{2^rd} \not\equiv -1 \mod{n}$ for $r \in \{0,1,2,...,s-1\}$

Then $n$ is not a prime,

When $a$ satisfies these equations $a$ is know as a witness for the composition of $n$.

When $a$ does not satisfy these equations and $n$ is composite, $a$ is known as a strong liar. This means $n$ is a strong probable prime to base $a$.

By randomly picking $a\in\{2,3,...,n-1\}$ multiple times it is unlikely for all of the selected $a$ to be strong liars.

This leads to the fact, if $p$ is $prime$, then the Miller Rabin test outputs $prime$. If p is $composite$, the algorithm outputs $composite$ but has a small chance of outputting $prime$.

In [ ]:
def random_prime(n_bit, t, trails=500):
    '''Returns a uniform n-bit prime, with probability at most 2^(-t) getting a composite,
    the higher the trails the less chance of failing to generate a prime but takes longer'''
    for i in range(0, trails):
        p = np.random.randint(0, 2, n_bit-2)
        b = "".join(str(i) for i in p)
        my_int = int("1" + b + "1", 2)  # Want at least n bit and gives odd
        if is_prime_miller_rabin(my_int, t):
            return my_int
    return "Fail"


def is_prime_miller_rabin(N, t=40):
    '''If N is prime, test always outputs True(prime)
    If N is composite outputs False(not prime) except with probability at most 2^(-t)'''

    if N == 2:
        return True  # Prime

    if N == 3:
        return True  # Prime

    if N % 2 == 0:
        return False  # Comp

    if N < 2:
        return False  # Comp

    d, s = calc_d_s(N)
    for j in range(0, t):
        a = ran.randrange(2, N-1)
        if is_composite(N, d, s, a):
            return False  # Comp
    return True  # Prime


def is_composite(N, d, s, a):
    '''Evaulate a^d != +-1 mod(N), and, a^(d*2^r) != -1 mod(N) for r in {1,2,...,s-1},
    However, if output False does not mean prime, it could be a strong liar'''
    remain = pow(a, d, N)
    if (remain != 1) and (remain != N - 1):
        for r in range(1, s):
            remain = pow(remain, 2, N)
            if remain != N-1:
                pass
            else:
                return False
    else:
        return False

    return True


def calc_d_s(N):
    s = 0
    Num = N - 1
    while Num % 2 == 0:
        Num //= 2
        s += 1
    d = Num
    return int(d), int(s)

In [ ]:
# Tests
assert calc_d_s(71)[0] == 35
assert calc_d_s(71)[1] == 1
assert calc_d_s(127)[0] == 63
assert calc_d_s(127)[1] == 1
assert calc_d_s(9627824804082068411)[0] == 4813912402041034205
assert calc_d_s(9627824804082068411)[1] == 1
assert calc_d_s(203981900391787397947)[0] == 101990950195893698973
assert calc_d_s(203981900391787397947)[1] == 1


N, d, s = 221, 55, 2
# 137 is a witness for compositeness of 221
a = 137
assert is_composite(N, d, s, a) == True

# 174 will lead to 221 being a prime or 174 is a strong liar for 221
a = 174
assert is_composite(N, d, s, a) == False

N, d, s = 71, 35, 1
a = 19
assert is_composite(N, d, s, a) == False


assert is_prime_miller_rabin(203981900391787397947, 40) == True
assert is_prime_miller_rabin(-1235, 40) == False
assert is_prime_miller_rabin(1, 40) == False
assert is_prime_miller_rabin(3, 40) == True

# Tests
expected_primes = np.array([2, 3, 5, 7, 11, 13, 17, 19])
expected_composites = np.array([1, 4, 6, 8, 9, 10, 12, 14, 15, 16, 18, 20])

primes, composites = get_primes_composites(20, is_prime_miller_rabin)
assert max(primes - expected_primes) == 0
assert max(composites - expected_composites) == 0

In [ ]:
def gen_modulas(n_bit):
    '''n_bit is the size of the primes you want, the secuirty level.
    This will output (N,p,q), N = p*q, p,q are n_bit uniform primes'''

    p = ""
    q = ""

    while type(p) == str or type(q) == str:
        p = random_prime(n_bit, 40)
        q = random_prime(n_bit, 40)
    N = p*q
    return N, p, q


def choose_e(phi_n):
    e = 2
    while True:
        if gcd(e, phi_n) == 1:
            return e
        else:
            e += 1


def gen_RSA(n_bit):
    '''n_bit is the size of the primes you want, the secuirty level.
    output, N,e,f, N = p*q, e is relatively prime to phi(N), f solves ef = 1mod(phi(N))'''
    N, p, q = gen_modulas(n_bit)
    phi_n = (p-1)*(q-1)
    e = choose_e(phi_n)
    f = modular_inverse(e, phi_n)
    return N, e, f

In [ ]:
assert choose_e(448) == 3

## The RSA-FDH Signature Scheme

This is from the hash and sign paradigm. Where FDH means full domain hash. There are three stages to this scheme: Generation, Sign and Verify
+ Generation
  - We input the n-bit size of the random uniform primes we want
  - This will give us a private key $(N,f)$ and public key $(N,e)$
  - Where $N=pq$ and
  > $ef \equiv 1 \mod{\phi{N}} $
  - A hash function $H$ is generated so all outputs are in $\{1,2,3,...,N-1\}$
  - The hash function $H$ is part of the public key
+ Sign
  - We input the private key $(N,f)$ and a message which will be hashed by $H$ into $h$
  - So we can compute the $s \in \mathbb{Z}$, which is the signature
  > $s \equiv m^d \pmod{N}$  
+ Verify
  - We input the public key $(N,e)$ and a message which will be hashed by $H$ into $h\prime$
  - We input $s$ the signature
  - Output $1$ which means authenticated if and only if
  > $h \equiv s^e \pmod{N}$

This works as
> $s^e \equiv (h^f)^e \equiv h^{ef \mod{\phi{N}}} \equiv h^1 \equiv h\mod{N}$

The reason why this is effective is because calculating $\phi{N}$ is very difficult, but with extra information $pq=N$ where $p,q \in$ Primes it makes $\phi{N}$ easy to calculate, this is know as a trap door. Additionally, the private key can be calculated from knowing the public key and the two primes that make up $N$.

Where $\phi{}$ is the Euler totient function. 

In [ ]:
def Generation(n_bit):
    '''input: n_bit, size of primes wanted
    output: private and public key'''
    N, e, f = gen_RSA(n_bit)
    private_key = {"N": N, "f": f}
    public_key = {"N": N, "e": e}
    return private_key, public_key


def Sign(message, private_key):
    h = my_hash(message, private_key["N"])
    s = pow(h, private_key["f"], private_key["N"])
    return s


def Verify(message, s, public_key):
    h_prime = my_hash(message, public_key["N"])
    h = pow(s, public_key["e"], public_key["N"])

    if h_prime == h:
        return 1  # Authenticated
    else:
        return 0  # Intruder

### Note, why is $\phi{(pq)}$ is easy to calculate

For large $N$, we have $\phi{(N)}$ is difficult to calculate. But since $\phi$ is a multiplicative function we have when $a,b$ are relatively prime ($gcd(a,b)=1$). Then,
> $\phi{(ab)} = \phi{(a)}\phi{(b)}$

We also have by the definition of the Euler totient function, for all $p\in primes$
> $\phi{(p)} = p-1$

This means if two primes denoted by $p,q$ can be found such that $N=pq$, then $\phi{(N)}$ can be quickly calculated by
> $\phi{(N)} = \phi{(p)}\phi{(q)} = (p-1)(q-1)$


Below is a graph of the totient function for the first 500 numbers

In [ ]:
x_prime, x_comp = get_primes_composites(500)

y_prime = np.array([totient(x_prime[i]) for i in range(0, len(x_prime))])
y_comp = np.array([totient(x_comp[i]) for i in range(0, len(x_comp))])

plt.figure(figsize=(14, 6))
plt.scatter(x_comp, y_comp, color="blue", label="Composites")
plt.scatter(x_prime, y_prime, color="red", label="Primes")
plt.title("Euler Totient Function")
plt.ylabel("Totient")
plt.xlabel("Number")
plt.legend(fontsize="x-large")
plt.show()

Below is also an Euler Totient graph but this time it is calculated with compiled code and on multiple threads thus using all CPU's. This means more numbers can be shown. 

In [ ]:
x_prime, x_comp = get_primes_composites(10000, is_prime_FUN=isprime_basic_fast)


y_prime = totient_fast_ufunc_par(x_prime)
y_comp = totient_fast_ufunc_par(x_comp)

plt.figure(figsize=(14, 6))
plt.scatter(x_comp, y_comp, color="blue", label="Composites", s = 1)
plt.scatter(x_prime, y_prime, color="red", label="Primes", s = 1)
plt.title("Euler Totient Function")
plt.ylabel("Totient")
plt.xlabel("Number")
plt.legend(fontsize="x-large")
plt.show()

### Note, forms of private key

If the private key is of the format $(p,q)$ where $N=pq$ given the public key $(N,e)$, the private key can be written in the format $(N,f)$ where $fe \equiv 1 \mod{\phi(N)}$ by the following code

In [ ]:
def convert_to_N_f(prime_private_key, public_key):
    '''inputs private key of form two primes and public key
    outputs the new private key of form(N,f)'''
    p = prime_private_key["p"]
    q = prime_private_key["q"]
    N = public_key["N"]
    e = public_key["e"]

    if p*q != N:
        return "The private key does not work with public key"

    phi_N = (p-1)*(q-1)
    f = modular_inverse(e, phi_N)
    new_private_key = {"N": N, "f": f}
    return new_private_key

In [ ]:
# Tests
prime_private_key = {"p": 41, "q": 61}
public_key = {"N": 2501, "e": 7}
private_key = convert_to_N_f(prime_private_key, public_key)
assert private_key == {"N": 2501, "f": 343}

## Examples of RSA Signature

### Example 1, no intruder

Bob wants to know the message he has been sent is from Alice. Therefore, Alice needs to generate private and public keys, sign the message. When Bob receives the message he has to verify the message is from Alice.

In [ ]:
message = "Hello Bob my freind"
# Generate keys with 512 bit primes
private_key, public_key = Generation(512)

# Alice needs to sign message
s = Sign(message, private_key)

# s and message are sent to Bob
# Bob should also have access to the public key
# Bob will now verify Alice sent the message
Verification = Verify(message, s, public_key)
Verification

As can be seen the verification process passed so Bob knows Alice sent the message.

Sometimes with very small probability the prime generator will produce a composite number. However, this is fine as then the message will fail the verification process and then the signature process happens again.

### Example 2, intruder Dave

Bob wants to know the message he has been sent is from Alice. And that Dave has not captured and changed the message. Same as before  Alice needs to generate private and public keys, sign the message. When Bob receives the message he has to verify the message is from Alice. If the message fails verification Dave has changed the message. 

In [ ]:
message = "Hello Bob my freind"
# Generate keys with 512 bit primes
private_key, public_key = Generation(512)

# Alice needs to sign message
s = Sign(message, private_key)

# s and message are sent to Bob
# Bob should also have access to the public key

# Dave intercepts the message and changes it
message = "I declare war on Bob"
# Dave sends new message to Bob

# Bob will now verify Alice sent the message
Verification = Verify(message, s, public_key)
Verification

The verification failed this means Bob knows that someone has intercepted and changed the message. 

### Example 3, 6 bit primes with tests and calculations

Let the message be hashed to $h$ where $h=15$, we will use 6 bit primes $p=41$ and $q=61$ so we get $N=2501$, we will use the fact that the Euler's totient function is multiplicative and $\phi{prime} = prime - 1$.
>$\phi{2501} = \phi{41}\phi{61} = 40*60 = 2400$

In [ ]:
assert totient(2501) == 2400

We will select $e=7$ as $gcd(7,2400)=1$

calc by hand show working
if test then we get all pass

In [ ]:
assert choose_e(2400) == 7

We need to solve for f
>\begin{align*}
ef &\equiv 1 \mod{\phi{N}}\\
7f &\equiv 1 \mod{2400}\\
f &= 343
\end{align*}

In [ ]:
assert modular_inverse(7, 2400) == 343

Therefore, we get private key $(N=2501, f=343)$ and public key $(N=2501, e=7)$

In [ ]:
private_key = {"N": 2501, "f": 343}
public_key = {"N": 2501, "e": 7}
message = 15

We need to sign the message, the signature $s$
>\begin{align*}
s &\equiv h^f \mod{N}\\
s &\equiv 15^{343} \mod{2501}\\
s &= 1053
\end{align*}

In [ ]:
s = Sign(message, private_key)
assert s == 1053

The message and signature are sent off to the person, the person will verify the message by checking if their $h\prime = h$.

Let us consider the case where the message is not changed.

>\begin{align*}
h &\equiv s^e \pmod{N}\\
h &\equiv 1053^{7} \pmod{2501}\\
h &= 15
\end{align*}

So since $h\prime = h$ the message will be verified.

In [ ]:
assert Verify(message, s, public_key) == 1

This time the same message is sent but a middle man intercepts the message and changes it. Let the new message be hashed to $h\prime=16$. Therefore, $h\prime \neq h$.


In [ ]:
message = 16
assert Verify(message, s, public_key) == 0

The middle man will struggle to forge $s$ as they will struggle to find $f$ as they know $e$ and $N$ but only knowing that information makes solving $ef \equiv 1 \mod{\phi{N}}$ for $f$ difficult for large $N$ as $\phi{N}$ takes a long time to calculate.

## Problems with the Plain RSA signature

The Plain RSA signature is the same as the RSA-FDH signature but without the hash function $H$. This means only integers in $\{1,2,...,N-1\}$ can be sent as the message. With this signature there are a few forms of attacks.

### A no control of message attack,

This attack only requires the public key $(N,e)$, it works as follows. The attacker uniformly picks a signature $s$ where $s\in\{1,2,3,...,N-1\}$. Then calculates the message $h$ where 
>$h \equiv s^e \mod{N}$

Therefore, the message $h$ has a valid signature $s$, this is a forgery as no signatures were signed by the owner of the public key. However, the attacker has no control over the messages $h$ that get forged.

If this attack is repeated multiple times for different values of the signature $s$ then the attacker can pick a forged message which suits their needs.

### Forging a signature on an arbitary message

Say the attacker wants to forge a message $h\in\{1,2,3,...,N-1\}$ with respect to the public key $(N,e)$, then the attacker picks  $h_1,h_2\in\{1,2,3,...,N-1\}$ which is different from $h$ with
> $h \equiv h_1h_2\mod{N}$

The attacker then needs to obtain valid signatures $s_1, s_2$ for $h_1, h_2$. Then we have $s \equiv s_1s_2 \mod{N}$ is a valid signature for $h$. This is because
>$s^e \equiv (s_1s_2)^e \equiv (h_{1}^{f}h_{2}^{f})^e \equiv h_1^{fe}h_2^{fe} \equiv h_1h_2 \equiv h \mod{N}$

where $f$ is part of the private key, but isn't needed for this to work.

This may not seem devastating but it is dangerous to make assumptions about what messages the signer may or may not be willing to sign.

Additionally, multiple valid signatures $\textbf{s} = \{s_1, s_2, ..., s_q\}$ and their corresponding messages $\textbf{h} = \{h_1,h_2,...,h_q\}$ can be captured. By using the same principle as above the attacker can obtain a valid signature on any of the $2^q-q$ other messages by taking products of the subset $\textbf{h}$.

Furthermore, one example of getting signatures and messages is when a client wants to connect to a server. If a client uses a signature scheme to authenticate with the server and this is done by the client signing a random message sent by the server. It is easy for a malicious server to ask for a certain message to be signed.

## Hash Function

Let $m$ be a string message and $h$ be a hashed message such that $H(m) = h$

This means to prevent these attacks the hash function $H$ must have certain properties.
+ $H$ must be hard to invert, this will prevent the "no control of message attack". As for each valid signature that is forged it is difficult to find the string message $m$ which corresponds to the hashed message $h$

+ $H$ must not have multiplicative relations, that means it is hard to find three messages $m,m_1,m_2$ such that $H(m) = H(m_1)H(m_2)$. This should prevent the "forgery of an arbitrary message".

+ It must be hard to find collisions for $H$. Since if $H(m_1)=H(m_2)$, then $m_1$ and $m_2$ have the same signature, this means forgery of a non signed message is trivial.


## Core 6: The Diffie-Hellman Protocol

### Sophie Germain Prime

First, before discussing the Diffie-Hellman protcol, we will be discussing what a Sophie Germain prime is and why it is useful in many situations for cryptography. Simply a Sophie Germain prime is a prime $q$ such that $p = 2q+1$ is also a prime which is called the complimentary safe prime of the $q$. The complimentary safe prime is very useful due to one of the properties it exhibits, to see this property we look at the prime decomposition of $p-1$.

> $p - 1 = (2q + 1) - 1 = 2q$

By doing this we already have the prime decompostion of $p-1$, it is simply $2q$. Now if you're wondering why this is useful, we must first discuss primitive roots. 

A primitive root of a prime $p$ is a value we will call $a$, where $a$ is an integer such that the smallest value $l$, known as the multiplicative order of $a$ relative to $p$, for which,

> $a^{l} \equiv 1mod(p)$

is true is:

> $l = p-1$

It was proven by the German mathmatician Gauss, that all primes have a primitive root and that the amount of primitive roots a prime has is equal to  $\phi{(p-1)}$, where  $\phi$ is Eulers totient, so we do not have to worry about a lack of primitive roots. Now a useful fact we can use is that the multiplicative order of $a mod(p)$ will always be a divisior of $p-1$. Thus we have found the use for a Sophie Germain prime, since we know the prime factorization of $p-1$ where $p$ is the complementary safe prime to some Sophie Germain prime $p$, we can use this to easily test if a value $a$ is a primitive root of the complementary safe prime. To do this all we have to do is check if for the safe prime p, any of the following are true:

>$a^{1} \equiv 1mod(p)$

>$a^{2} \equiv 1mod(p)$

>$a^{\frac{(p-1)}{2}} \equiv 1mod(p)$

We only have to check this since we know the multiplicative order of $a mod(p)$ is a divisor of $2q$, so the multiplicative order must be $1,2$ or $q$, hence if none of the above are true, then the multiplicative order is $2q$ which means $a$ is a primitive root of our safe prime. This will get us started on the Diffie-Hellman protocol, so first we write a function to get Sophie Germain primes, and their complementary safe primes along with a function to get primitive roots for complementary safe primes.

In [6]:
def sgrandprime(n_bit,t): 
    """This function returns a Sophie Germain prime, and its complementary safe prime."""
    while True: #Keeps searching untill we find a Sophie Germain prime
        bits = False
        while bits == False:
            p = random_prime(n_bit-1, t, trails = 500) # We generate random primes here
            if (2*p+1).bit_length() == n_bit:#We make sure the safe prime is of the desired bit length
                bits = True 
        if is_prime_miller_rabin((2*p)+1,t = 40): #Check if the random prime is Sophie germain
            return p, (2*p)+1



def safe_primitive_root(publicp):
    """This will return a primitive root for a given complementary safe prime
    for some Sophier Germain prime"""
    q = publicp[1] #Safe prime
    p = publicp[0] #Sophie germain prime
    b = 2 #Start checking for primitive roots starting from a value of 2
    primitive = False
    while not primitive: #Keeps searching untill primitive root is found
        if b%q == 1 or (b*b)%q == 1 or pow(b,p,q) == 1:   #if any of these are true, b is not a primitive root
            b += 1
            continue
        else: #if none are true, then b is a primitive root.
            primitive = True
    return b
    


NameError: name 'random_prime' is not defined

### The Deffie-Hellman Protocol - First Steps

Before discussing the first steps of the Diffie-Hellman protocol, we will discuss what the protocol does. This protocol allows two or more parties (In cryptography the first two parties are usually referred to as Alice and Bob) to generate a shared secret key that each party knows, and has access to, without ever having to communicate the key in any form. The importance here is that the secret key is not communicated between parties, because if it is, the key is then suseptible to being observed by an outside party. Thus from this it is easy to see why the Deffie-Hellman protocol is so useful. An example of one of the main uses of this is that it allows parties say Alice and Bob to send encrypted messages to each other using the secret key and then decrypt these message using the same secret key. Hence if an outside party was to observe one of these messages, without the secret key they will not be able to read the message (at least without knowing how they encrypted the message and any potential cracks for this method).

So how does the Diffie-Hellman protocol work? First we will look at the two party situation, and we will call these parties Alice and Bob. To begin we need both Alice and Bob to have access to a very large safe prime $p$ (when discussing this protcol we will be using a 256 bit prime) and a primtive root of the prime we will call $g$, however unlike the secret key, $p$ and $g$ can be made public for anyone to have access to. Now when dicussing how the protocol works, you may soon notice that the safe prime Bob and Alice make public does not need to be a safe prime or even a prime at all and the primitive root can be any number for the protocol to work. Despite this we use a safe prime and a primitive root because hackers can calculate discrete logarithms (for example by using the Pohlig-Hellman protocol) to work out the secret key, however the algorithms hackers use is much faster if the number we make public here is not a safe prime, thus for security purposes we use a safe prime and one of the primitive roots for this prime. Hence by using our "sgrandprime" function above, we can generate a safe prime for Alice and Bob to use.

Now in the next step we want Alice and Bob to generate a private key for themselves, they can not tell each other about these keys nor anyone else, this is for them and only them to know and have access to. This private key is a random number, for alice we will call her private key $a$ and for Bob $b$, however the larger the private key again the better as it makes it harder for hackers to get access to the secret key so again we generate 256 bit random numbers. 

Below we will generate the safe prime $p$ and primitive root $g$ using the functions from before, also we will make a function to generate our the private keys $a$ and $b$.

In [ ]:

def Private_keys(n):
    """This function simply"""
    keys = []
    for i in range (0,n):
        bits = False
        while bits == False:
            x = ran.SystemRandom().getrandbits(256)
            if x.bit_length() == 256:
                keys.append(x)
                bits = True
    return keys

Publicprimes = sgrandprime(256,40)#Generate the primes to be used
p = Publicprimes[1]
g = safe_primitive_root(Publicprimes) # Get the primitive root of p
keys = Private_keys(2) #Generate the private keys for Alice and Bob (In a real scenorio, they will generate their key alone.)
a = keys[0]
b = keys[1]

### The Diffie-Hellman protocol - Generating Secret Key

Once Alice and Bob have their private keys, their next step is to generate their public keys. We will call the public key Alice generates $A$ and Bobs $B$. To generate these public keys we do the following calculations:

>$A = g^{a}mod(p)$

>$B = g^{b}mod(p)$

After we complete these calculations, we now have the public keys for Alice and Bob. The next step is for Alice to send her public key $A$ to Bob, and for Bob to send $B$ to Alice. Though this seems like a compromising step for a protocol based around security, for a hacker to use the public keys to get the private keys it is very difficult because $p$ is a safe prime and $g$ is a primitive root for $p$. By using these numbers a hacker has a very small amount of attack vectors when trying to find the private keys.

Once Alice has Bobs public key and Bob has Alices, we calculate the secret key that we will now call $S$. When calculating the secret key this must be done seperately by both Bob and Alice in private as to not let anyone gain access to the secret key. For Alice to generate the $S$ she must do the following calculation:

>$S = B^{a}$

and Bob must generate $S$ by calculating:

>$S = A^{b}$

This at first look may not be entirely obvious as to why the calculation Alice does calculates the same secret key as Bobs calculations. However this is due to simple properties of modulo exponents, as you can see below:

>$S = B^{a} = (g^{b}mod(p))^{a} = g^{ba}mod(p) = (g^{a}mod(p))^{b} = A^{b} = S$

However the calculations above only apply to a two party system. Luckily for us this generalizes quite simply for an $n$ party system. As before we have our safe prime $p$ and a primitive root of $p$ we call $g$, and all $n$ parties must generate their own private key in the same way as before and their relative public key in the same way as before. The only difference for an $n$ party system comes when calculating the secret key $S$. To do this let us label the n private keys generated as below:

> $P_{1}, P_{2}, ... , P_{n}$

Now all n parties have already calculated their relative public keys, which are as below:

> $g^{P_{1}}mod(p), g^{P_{2}}mod(p), ... , g^{P_{n}}mod(p)$

We want our final secret key $S$ to be of the same form before, meaning we have $g^{x}mod(p)$, where $x$ is the product of all private keys involved. Hence to achieve this in an n party case, we first label each party $1,2,..n$ and the $i^{th}$ party must send their public key to the $(i+1)^{th}$ party also party $n$ must give his public key to party $1$. Then each member must generate their new public key as below:

> $(g^{P_{n}}mod(p))^{P_{1}}, g^{P_{1}}mod(p))^{P_{2}}, ... , g^{P_{n-1}}mod(p))^{P_{n}}$

Once this has been completed, we again get each party to send their new public key to the party ahead of them. We repeat this process a total of $n-1$ times, and the final calculation will give each party the secret key $S$ which for the $i{th}$ party will be of the form:

> $(g^{P_{i-1}P_{i-2}...P_{1}P_{n}...P_{i+1}}mod(p))^{P_{i}} = g^{P_{1}P_{2}...P_{n}}mod(p) = S$

After all parties have the secret key $S$, they can send each other messages encrypted using the key $S$ and decrypt the messages they receive from one another. Below are the functions we use to achieve this.

In [ ]:
def Public_Keys(primiroot,Privates,Prime):
    """This function completed the neccessary calculation to calculate
    the Public keys for given private keys and safe prime"""
    n = len(Privates) #So we know how many Public keys we must generate 
    Publics = []
    for i in range (0,n):
        Publics.append(pow(primiroot,Privates[i],Prime)) #Calculation required to generate public keys
    return Publics 

Publics = Public_Keys(g,keys,p) #Generate the public keys for Alice and Bob 

A = Publics[0]
B = Publics[1]

def Secret_key_steps(publics, privates, generator):
    """This function completes all neccessary public key swaps between parties
    so that they all end up with the same Secret Key"""
    if str(publics).isdigit() == True: #This is to check if "publics" is one single public key (and not a list of many public
                                       # keys for the general n parties) and if yes then we make "publics" a list of
                                       # one key only
        publics = [publics]
    if str(privates).isdigit() == True:#This is to check if "privates" is one single private key (and not a list of many private
                                       # keys for the general n parties) and if yes then we make "privates" a list of
                                       # one key only
        privates = [privates]
    n = len(publics)
    if n == 1: #if n == 1 this means it is one person using the public key they have been given to calculate the next public
               # key or the final secret key, so we simple calclate this below.
        return(pow(publics[0], privates[0], generator))
    for m in range(0,n-1):  #For a party of length n, only have to complete this process n-1 times as mentioned above
        Temp = [] #Stores the current public key
        for i in range(0,n):
            Temp.append(pow(publics[(i-1)%n], privates[i], generator) )#Calculates current public key
        for j in range(0,n):    
            publics[j] = Temp[j]
    Final = Temp #After the final repeated is complete "Temp" will be equal to the secret key
    return Final

Alice_S = Secret_key_steps(B,a,p)#Alices calculation for the Secret key
Bob_S = Secret_key_steps(A,b,p)#Bobs calculation for the Secret key

print("Alice calculates the secret key to be",Alice_S,", and Bob calculates the secret key to be", Bob_S, ". Hence they both have the same secret key")

### Alice and Bob - Sending a Message

Now that Alice and Bob have both calculated the secret key independently without ever communicating the key with one anoher, they both now have access to a key which they can use to encrypt and decrypt messages they send to each other. Below is an example of how they can do this.


In [ ]:
#Below is an example of Alice encrypting a message using the Vigenere cipher using the calculated secret key
Alice_message = Vigenere_cipher_ext("Hello Bob, I'm sending this message to you with the hopes that you have calculated the secret key, so that we can now communicate in private", str(Alice_S), "encrypt")
#Below is how Bob will decrypt Alices message using the secret key
Bob_decryption = Vigenere_cipher_ext(Alice_message, str(Bob_S), "decrypt")

print("Thus Alice has encrypted her message using the key, to get the encrypted message - ", Alice_message,".Bob now decrypts this message using the secret key to get -", Bob_decryption)

### Three Party Example

As I mentioned above, the functions we created were generalized for n people, so to carry out this protocol for a 3 party scenorio is very simple. Below is an example of our code working for 3 partys.

In [ ]:
#Example of this being used to generate a secret key for a 3 party system

threekeys = Private_keys(3) #These will be the 3 private keys for our parties, however they would in a real scenorio
                            #be computed alone on each persons computer

Three_Alice_private = threekeys[0]
Three_Bob_private = threekeys[1]
Three_Charlie_private = threekeys[2]

Three_Public = Public_Keys(pr,threekeys,p) #Now they all calculate their intial public key 

Three_Alice_public1 = Three_Public[0] 
Three_Bob_public1 = Three_Public[1]
Three_Charlie_public1 = Three_Public[2]

General_function_keys = Secret_key_steps(Three_Public,threekeys,p) #This does the final step where each public key is passed to each party
                                                   #as many times as is needed for all parties to have the same secret key
                                                   #however again, this would in a real scenorio be done multiple times 
                                                   #alone.

#However if we were to do this manually alone as in a real scenorio, we will follow what each party must do below:

#First each party must pass their public key to the next part in the line

#Below each party must now calculate their second public key and pass this to the next party to calculate the secret key

Three_Alice_public2 = Secret_key_steps(Three_Charlie_public1,Three_Alice_private,p)
Three_Bob_public2 = Secret_key_steps(Three_Alice_public1,Three_Bob_private,p)
Three_Charlie_public2 = Secret_key_steps(Three_Bob_public1,Three_Charlie_private,p)

#Now as this is only a 3 party scenorio, we have one final step to calculate the Secret key below:

Three_Alice_S = Secret_key_steps(Three_Charlie_public2,Three_Alice_private,p)
Three_Bob_S = Secret_key_steps(Three_Alice_public2,Three_Bob_private,p)
Three_Charlie_S = Secret_key_steps(Three_Bob_public2,Three_Charlie_private,p)

print("The keys we generated using our general function were:")
print(General_function_keys)
print("The independelty calculated keys were:")
print("Alices S -", Three_Alice_S)
print("Bobs S -", Three_Bob_S)
print("Charlies S -", Three_Charlie_S)
print("As you can see their all equal, whever we use our general function, or do it indepently")

### The Diffie-Hellman protocol - Function notes

The functions we have created above are generalized functions as I mentioned before, hence they are good for show casing how the protocol works for both a 2 party system and for an n party system. However as mentioned when implementing the The Diffie-Hellman protocol in a real world scenorio these calculations should be done in private for one person. This means you should not use the functions as I have above to calculate private keys, public keys and the secret key for each party in the system on one computer. Instead you will use the functions to only calculate everything for yourself in privacy, as if everything was done from one computer, this would negate the entire process.

## Mark Part

Solving Discrete Logarithm Problem Using Brute Force

In [ ]:
Private_keys(1)

In [ ]:
def discrete_logarithm_brute(a, N, z):
    """Solves the discrete logarithm problem

    Solves z = a^b mod(N) for b, by using a brute force method 

    Parameters
    ----------
    a : int
    N : int
    z : int

    Returns
    -------
    int
        Solves z = a^b mod(N) for b,
    """

    b = 1
    while True:
        if pow(a, b, N) == z:
            return b
        b += 1


def discrete_logarithm_brute_better(a, N, z):
    """Solves the discrete logarithm problem

    Solves z = a^b mod(N) for b, by using a brute force method 

    Parameters
    ----------
    a : int
    N : int
    z : int

    Returns
    -------
    int
        Solves z = a^b mod(N) for b,
    """
    b = 1
    a_new = a
    while True:
        if a_new == z:
            return b
        a_new = (a_new*a) % N
        b += 1

@numba.jit('int64(int64,int64,int64)', nopython = True)
def discrete_logarithm_brute_better_fast(a, N, z):
    """Solves the discrete logarithm problem

    Solves z = a^b mod(N) for b, by using a brute force method, uses a complied version

    Parameters
    ----------
    a : int
    N : int
    z : int

    Returns
    -------
    int
        Solves z = a^b mod(N) for b,
    """
    b = 1
    a_new = a
    while True:
        if a_new == z:
            return b
        a_new = (a_new*a) % N
        b += 1


In [ ]:
# Tests
assert discrete_logarithm_brute(15, 2342, 167) == 37
assert discrete_logarithm_brute(5, 352, 133) == 25
assert discrete_logarithm_brute(3452, 123456, 39104) == 543

# Tests
assert discrete_logarithm_brute_better(15, 2342, 167) == 37
assert discrete_logarithm_brute_better(5, 352, 133) == 25
assert discrete_logarithm_brute_better(3452, 123456, 39104) == 543

# Tests
assert discrete_logarithm_brute_better_fast(15, 2342, 167) == 37
assert discrete_logarithm_brute_better_fast(5, 352, 133) == 25
assert discrete_logarithm_brute_better_fast(3452, 123456, 39104) == 543

In [ ]:
%timeit discrete_logarithm_brute(3452, 123456, 39104)
%timeit discrete_logarithm_brute_better(3452, 123456, 39104)
%timeit discrete_logarithm_brute_better_fast(3452, 123456, 39104)

In [ ]:
#discrete_logarithm_brute(2,4,3)
#Will run forever

In [ ]:
def gen_public_key(n_bit):
    """generates public information

    Generates then calculates public information

    Parameters
    ----------
    n_bit : int
        The size of the p prime wanted

    Returns
    -------
    dict
        The public information
        A : solution to g^a mod(p)
        g : primitive root
        p : large n_bit prime

    """
    q, p = sgrandprime(n_bit, 40)
    g = safe_primitive_root([q, p])
    a = Private_keys(1, n_bit)[0]
    A = pow(g, a, p)
    #print("a: " + str(a) + " A: " + str(A) + " g: " + str(g) + " p: " + str(p))
    return {"A": A, "g": g, "p": p}


def time_func(Func, num_samples, Func_parameters = [[]]):
    """Times the function

    Parameters
    ----------
    Func : function
        Function that is being timed 
    num_samples : int
        Number of times Func will run
    Func_parameters : 2D array
        Each array in 2D array are the parameters of the Function Func, if num_samples
        greater than number of arrays then it will loop over
    
    Returns
    -------
    tuple
        mean_time,
        std_time
        
        
    Examples
    --------
    time_func(decompose, 4, [[3226322291321]])   
    #will time decompose function 4 times with parameter 3226322291321
       
    time_func(decompose, 3, [[3304575252941], [1954377239027], [2964234252593]])   
    #will time decompose function 3 times with a differenc parameter each time
    
    
    """
    times = np.zeros(num_samples)
    loopover = len(Func_parameters)
    for i in range(num_samples):
        temp_start_time = time.time()
        Func(*Func_parameters[i%loopover])
        temp_finish_time = time.time()
        times[i] = (temp_finish_time - temp_start_time)
    
    return times.mean(), times.std()
    

In [ ]:
def format_input_parameters(public_keys, my_order = ["g", "p", "A"]):
    """Formats parameters for the function


    Parameters
    ----------
    public_keys : array of dict

    my_oreder : array of str
        What order is wanted for the output

    Returns
    -------
    2D array
        The parameters for the funciton

    """

    formated_public_keys = []
    
    for public_key in public_keys:
        temp_public_key = []
        for key in my_order:
            temp_public_key.append(public_key[key])
        formated_public_keys.append(temp_public_key)

    return formated_public_keys



In [ ]:
# Test format_input_parameters
input_public_keys = [{"A": 307, "g": 5, "p": 863},
                     {"g": 5, "A": 844, "p": 983},
                     {"p": 563, "g": 2, "A": 83}]
expected_format = [[5, 863, 307],
                   [5, 983, 844],
                   [2, 563, 83]]
assert format_input_parameters(input_public_keys) == expected_format

In [ ]:
start = 6
stop = 28
num_samples = 10

mean_times = []
std_times = []

for n_bit in range(start, stop):
    public_keys = [gen_public_key(n_bit) for i in range(num_samples)]

    input_parameters = format_input_parameters(public_keys)
    temp_mean, temp_std = time_func(
        discrete_logarithm_brute_better_fast, num_samples, input_parameters)
    mean_times.append(temp_mean)
    std_times.append(temp_std)

print("discrete_logarithm_brute(g, p, A)")

my_time_data = {"Mean time": mean_times,
                "Std time": std_times}
time_data_frame = pd.DataFrame(data=my_time_data)
time_data_frame.index = range(start, stop)
time_data_frame.index.name = "bit size of p and a"

print(time_data_frame)

In [ ]:
mean_times

In [ ]:
std_times

In [ ]:
# Silly example data
#bp_x = np.linspace(0, 2*np.pi, num=40, endpoint=True)
#bp_y = np.sin(bp_x)


# Make the plot
plt.plot(range(start,stop),mean_times, linewidth=3, linestyle="--",
         color="blue", label=r"Collected data")
plt.xlabel(r"$n$ bit size of p and a")
plt.ylabel(r" Mean time (seconds)")
plt.title(r"Discrete Logarithm Brute Force Time")
#plt.xlim(0, 2*np.pi)
#plt.ylim(-1.1, 1.1)
plt.legend(loc="upper left")
plt.show()



In [ ]:
q, p = sgrandprime(10, 40)
g = safe_primitive_root([q, p])
a = Private_keys(1, 10)[0]
print(a)
print(p)
A = pow(g, a, p)
print(A)
# A = g^a mod(p)

discrete_logarithm_brute(g, p, A)

In [ ]:
# File save
# Will be frozen so can't run when montoring
start = 6
stop = 70
num_samples = 25


file_name = "BruteForceDiscreteLogData.csv"


for n_bit in range(start, stop):
    public_keys = [gen_public_key(n_bit) for i in range(num_samples)]

    input_parameters = format_input_parameters(public_keys)
    temp_mean, temp_std = time_func(
        discrete_logarithm_brute, num_samples, input_parameters)

    string_to_append = str(n_bit) + ", "
    string_to_append += str(temp_mean) + ", "
    string_to_append += str(temp_std) + ", "
    string_to_append += str(num_samples) + "\n"
    brute_force_file = open(file_name, "a")
    brute_force_file.write(string_to_append)
    brute_force_file.close()